<a href="https://colab.research.google.com/github/Asmina-hub/AGI_Project/blob/main/Image_overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a simple project in which we build a simple NN with just1 hidden layer to check how it performs on Image classification.

In [ ]:
!pip install torch gcsfs
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import gcsfs
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
from tqdm import tqdm
!pip install wandb
import wandb
wandb.login()
import pprint

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: asminanassar77 (asminanassar77-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
sweep_config = {
 'method': 'grid'
}
metric = {
    'name': 'val_acc',
    'goal':'maximize'
}
sweep_config['metric'] = metric
parameters_dict = {
    'epoch' : {'values': [5,10]},
    'hidden_layer': {'values': [64,128]},
    'learning_rate': {'values': [0.001,0.0001]},
    'batch_size': {'values': [64,128]},
    'image_width': {'values': [224,64]},
    'experiments' : {'values': ['all'] },

}
sweep_config['parameters'] = parameters_dict

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'val_acc'},
 'parameters': {'batch_size': {'values': [64, 128]},
                'epoch': {'values': [5, 10]},
                'experiments': {'values': ['all']},
                'hidden_layer': {'values': [64, 128]},
                'image_width': {'values': [224, 64]},
                'learning_rate': {'values': [0.001, 0.0001]}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config,project='flower image classification')

Create sweep with ID: s8mha5oc
Sweep URL: https://wandb.ai/asminanassar77-/flower%20image%20classification/sweeps/s8mha5oc


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
CLASS_NAMES=['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
CLASS_INDEX = {name:i for i,name in enumerate(CLASS_NAMES)}
def train():
  with wandb.init() as run:
    config = wandb.config
    IMAGE_WIDTH = config.image_width
    IMAGE_HEIGHT = config.image_width
    IMAGE_RGB = 3
    EPOCH = config.epoch
    LR = config.learning_rate
    BATCH_SIZE = config.batch_size
    HIDDEN_LAYER = config.hidden_layer

    transform = transforms.Compose([transforms.Resize((IMAGE_HEIGHT,IMAGE_WIDTH)),transforms.ToTensor()])
    train_dataset = FlowerDataSet("gs://cloud-ml-data/img/flower_photos/train_set.csv",
                              transform=transform )
    test_dataset = FlowerDataSet("gs://cloud-ml-data/img/flower_photos/eval_set.csv",
                                transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)


    model=ShallowNeuralNetwork(len(CLASS_NAMES),IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_RGB,HIDDEN_LAYER,config).to(Device)

    train_epoch(LR,EPOCH,model, train_loader, test_loader)

In [ ]:

    class ShallowNeuralNetwork(nn.Module):
      def __init__(self,num_class,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_RGB,HIDDEN_LAYER,config):
        super(ShallowNeuralNetwork, self).__init__()
        self.flatten=nn.Flatten()
        if config.experiments is None:
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(IMAGE_HEIGHT * IMAGE_WIDTH * IMAGE_RGB, HIDDEN_LAYER),
              nn.ReLU(),
              nn.Linear(HIDDEN_LAYER, num_class)
          )
        elif config.experiments == 'batchnorm':
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(IMAGE_HEIGHT * IMAGE_WIDTH * IMAGE_RGB, HIDDEN_LAYER),
              nn.BatchNorm1d(HIDDEN_LAYER),
              nn.ReLU(),
              nn.Linear(HIDDEN_LAYER, num_class)
          )
        elif config.experiments == 'dropout':
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(IMAGE_HEIGHT * IMAGE_WIDTH * IMAGE_RGB, HIDDEN_LAYER),
              nn.ReLU(),
              nn.Dropout(0.5),
              nn.Linear(HIDDEN_LAYER, num_class))
        elif config.experiments == 'all':
            self.linear_relu_stack = nn.Sequential(
              nn.Linear(IMAGE_HEIGHT * IMAGE_WIDTH * IMAGE_RGB, HIDDEN_LAYER),
              nn.BatchNorm1d(HIDDEN_LAYER),
              nn.ReLU(),
              nn.Dropout(0.5),
              nn.Linear(HIDDEN_LAYER, num_class)
          )

      def forward(self, x):
        # Flatten the image data in a better way
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits



In [ ]:
#cutom class in pytorch to load the dataset
class FlowerDataSet(Dataset):
  def __init__(self,csv_file_path,transform=None):
    self.fs = gcsfs.GCSFileSystem()
    with self.fs.open(csv_file_path, 'rb') as f:
      self.data=pd.read_csv(f,header=None, names=['filename','label'])
    self.transform=transform
  def __len__(self):
    return self.data.shape[0]
  def __getitem__(self,index):
    image_path=self.data.iloc[index,0]
    label_str = self.data.iloc[index,1]
    # Use gcsfs to open the image file
    with self.fs.open(image_path, 'rb') as f:
      image=Image.open(f).convert( 'RGB')

    if self.transform:
      image=self.transform(image)
    label = CLASS_INDEX[label_str]

    return image,label

In [ ]:
Device = 'cuda' if torch.cuda.is_available() else 'cpu'
def train_epoch(LR,EPOCH,model, train_loader, test_loader): # Add train_loader and test_loader as arguments
  train_losses, val_losses = [], []
  train_accuracies, val_accuracies = [], []
  criterion = nn.CrossEntropyLoss()
  optimizer = Adam(model.parameters(), lr=LR,weight_decay=0.01) #weight decay is equalant to L2 regularization
  # Remove wandb.init() from here
  for epoch in range(EPOCH): # Loop over epochs here
    model.train()
    running_loss = 0.0
    train_correct, train_total = 0, 0
    for batch_idx, (images, labels) in enumerate(train_loader):
      images, labels= images.to(Device), labels.to(Device)
      optimizer.zero_grad()
      outputs=model(images)
      loss=criterion(outputs,labels)
      loss.backward()
      optimizer.step()

      running_loss+=loss.item()
      _, predicted = torch.max(outputs, 1)
      train_total += labels.size(0)
      train_correct += (predicted == labels).sum().item()

      avg_loss = running_loss / (batch_idx + 1)
      avg_acc = 100.0 * train_correct / train_total


    train_acc = 100.0 * train_correct / train_total
    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(train_acc)

    # ---- Validation ----
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(Device), labels.to(Device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100.0 * val_correct / val_total
    avg_val_loss = val_loss / len(test_loader)
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_acc)

    # Log metrics to wandb
    wandb.log({ # Use wandb.log here
        "epoch": epoch,
        "train_loss": train_losses[-1],
        "train_acc": train_accuracies[-1],
        "val_loss": val_losses[-1],
        "val_acc": val_accuracies[-1]
    })

In [ ]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: zv56qk4w with config:
wandb: 	batch_size: 64
wandb: 	epoch: 5
wandb: 	experiments: all
wandb: 	hidden_layer: 64
wandb: 	image_width: 64
wandb: 	learning_rate: 0.0001


epoch,▁▃▅▆█
train_acc,▁▅▆▇█
train_loss,█▄▃▂▁
val_acc,▁▅█▅▆
val_loss,█▂▄▅▁
epoch,4
train_acc,52.60606
train_loss,1.18974
val_acc,50.54054
val_loss,1.20272


wandb: Agent Starting Run: elcpqrk8 with config:
wandb: 	batch_size: 64
wandb: 	epoch: 5
wandb: 	experiments: all
wandb: 	hidden_layer: 128
wandb: 	image_width: 224
wandb: 	learning_rate: 0.001


epoch,▁▃▅▆█
train_acc,▁▅▇██
train_loss,█▄▂▁▁
val_acc,▁▃█▇▃
val_loss,█▃▁▁▄
epoch,4
train_acc,45.81818
train_loss,1.27425
val_acc,40.54054
val_loss,1.33722


wandb: Agent Starting Run: bmiae10m with config:
wandb: 	batch_size: 64
wandb: 	epoch: 5
wandb: 	experiments: all
wandb: 	hidden_layer: 128
wandb: 	image_width: 224
wandb: 	learning_rate: 0.0001


epoch,▁▃▅▆█
train_acc,▁▄▅▇█
train_loss,█▅▃▂▁
val_acc,▁▆▇█▅
val_loss,█▁▂▂█
epoch,4
train_acc,57.12121
train_loss,1.0917
val_acc,47.2973
val_loss,1.2619


wandb: Agent Starting Run: 9laxz46v with config:
wandb: 	batch_size: 64
wandb: 	epoch: 5
wandb: 	experiments: all
wandb: 	hidden_layer: 128
wandb: 	image_width: 64
wandb: 	learning_rate: 0.001
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


epoch,▁▃▅▆█
train_acc,▁▅███
train_loss,█▄▂▂▁
val_acc,▇▇▆▁█
val_loss,▃▃▄█▁
epoch,4
train_acc,48.63636
train_loss,1.2067
val_acc,47.02703
val_loss,1.20478


wandb: Agent Starting Run: 865e5j06 with config:
wandb: 	batch_size: 64
wandb: 	epoch: 5
wandb: 	experiments: all
wandb: 	hidden_layer: 128
wandb: 	image_width: 64
wandb: 	learning_rate: 0.0001


epoch,▁▃▅▆█
train_acc,▁▄▆▇█
train_loss,█▄▃▂▁
val_acc,▁▅▅▆█
val_loss,█▅▅▂▁
epoch,4
train_acc,57.30303
train_loss,1.10012
val_acc,51.35135
val_loss,1.19545


In [ ]:
class EarlyStopping:
  def __init__(self, patience=5 , )
